All import need to run the code

In [16]:
import random

from matplotlib import pyplot as plt
import cv2
import numpy as np
from tensorflow import keras

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

Load dataset from keras cifar10 - https://keras.io/api/datasets/cifar10/

In [17]:
def load_dataset(_amount_of_data=1, _test_ratio=0.2):

    (_training_classes, _training_labels), (_test_classes, _test_labels) = keras.datasets.cifar10.load_data()
    _training_classes, _test_classes = _training_classes / 255.0, _test_classes / 255.0


    _training_classes = _training_classes[:int((_amount_of_data * (1-_test_ratio)))]
    _training_labels = _training_labels[:int((_amount_of_data * (1-_test_ratio)))]
    _test_classes = _test_classes[:int((_amount_of_data * _test_ratio))]
    _test_labels = _test_labels[:int((_amount_of_data * _test_ratio))]

    _training_classes = flatten_array(_training_classes)
    _test_classes = flatten_array(_test_classes)

    return (_training_classes, _training_labels), (_test_classes, _test_labels)

Simple function to rescale the images in the dataset.

In [18]:
def rescale_data(_data):
    for pic in range(len(_data)):
       _data[pic]  = cv2.resize(_data[pic], (32,32))
    return _data

Simple function to plot an image using matplotlib, pyplot as plt

In [19]:
def plot_image(_classes):
    plt.figure(figsize=(1.5, 1.5))
    plt.imshow(_classes.reshape(32,32,3))
    plt.show()
    plt.close()

Function to implement bagging for the given dataset. - To read more about bagging, follow this link: https://www.ibm.com/cloud/learn/bagging#:~:text=Bagging%2C%20also%20known%20as%20bootstrap,be%20chosen%20more%20than%20once.

In [20]:
def bagging(_data, _labels, _number_of_bags=3):
    _new_lst = []

    _current_bag = 0
    while _current_bag < _number_of_bags:

        _new_lst.append([])

        _bag_classes = []
        _bag_labels = []

        _count = 0
        while _count < len(_data):
            _index = random.randint(0, len(_data) - 1)
            _bag_classes.append(_data[_index])
            _bag_labels.append(_labels[_index])
            _count += 1

        _new_lst[_current_bag].append(np.array(_bag_classes))
        _new_lst[_current_bag].append(np.array(_bag_labels))

        _current_bag += 1
    return _new_lst

Simple function to flatten an array.

In [21]:
def flatten_array(_array):
    _index = 0
    _return_lst = []

    for item in _array:
        _return_lst.append(item.flatten())
        _index += 1
    return np.array(_return_lst)

Support vector machine classification implementation

In [ ]:
def SVC(_amount_of_data=100, _use_optimization = False):
    (_training_classes, _training_labels), (_test_classes, _test_labels) = load_dataset(_amount_of_data=_amount_of_data)

    _training_labels = _training_labels.flatten()
    _test_labels = _test_labels.flatten()

    if _use_optimization:   # Gridsearch, very heavy on the computational side.
        param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1],'kernel':['rbf','poly']}
    else:
        param_grid = {'C':[0.1],'gamma':[0.01],'kernel':['rbf']}

    svc=svm.SVC(probability=True)
    model=GridSearchCV(svc, param_grid)

    model.fit(_training_classes, _training_labels)
    y_pred=model.predict(_test_classes)

    print(f"The model is {accuracy_score(y_pred,_test_labels)*100}% accurate")
    return accuracy_score(y_pred,_test_labels)*100

Function to easily run the SVC. Set parameters as you want. It's recommended to read the readme.md file in the GitHub project.

In [ ]:
def run_multiple_times(_runs=1, _amount_of_data=1000, _use_optimization=False):
    _run = 0
    _average = 0

    while _run < _runs:
        _average += SVC(_amount_of_data=_amount_of_data, _use_optimization=_use_optimization)
        _run += 1

    print("Runs completed!")
    print(" ")
    print("Average score: " + str(_average / _runs))

run_multiple_times(_runs=1, _amount_of_data=500, _use_optimization=True)